In [ ]:
!apt-get update -y
!apt-get install python3-openslide -y
!pip install openslide-python
!pip install -U tensorboard_plugin_profile

In [ ]:
import tensorflow as tf
import tensorflow_reader as tfr
from datetime import datetime
from packaging import version
import os

In [ ]:
print("TensorFlow version: ", tf.__version__)
device_name = tf.test.gpu_device_name()
if not device_name:
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logs, histogram_freq=1)

In [ ]:
if False:
    # Example code that isn't working in our case.
    summary_writer = tf.summary.create_file_writer(logs)
    with summary_writer.as_default():
        model, batched_dist, strategy = tfr.readExample()
        features, metadata = tfr.predictExample(model, batched_dist, strategy)
        tfr.outputExample(features, metadata)

In [ ]:
tf.profiler.experimental.start(logs)

In [ ]:
devices, strategy, model = tfr.strategyExample()

In [ ]:
model, batched_dist, strategy = tfr.readExample(devices, strategy, model)

In [ ]:
features, metadata = tfr.predictExample(model, batched_dist, strategy)

In [ ]:
tfr.outputExample(features, metadata)

In [ ]:
tf.profiler.experimental.stop()

In [ ]:
print(logs)
%load_ext tensorboard
%tensorboard --logdir=logs

In [ ]:
del model, batched_dist, strategy, features, metadata